# LangChain


LangChain provides several kinds of 'memory' classes to store and accumulate the conversation, some of them are:

* [ConversationBufferMemory](#ConversationBufferMemory)
* [ConversationBufferWindowMemory](#ConversationBufferWindowMemory)
* [ConversationTokenBufferMemory](#ConversationTokenBufferMemory)
* [ConversationSummaryMemory](#ConversationSummaryMemory)

LLM are stateless where each transaction is independent.

## ConversationBufferMemory

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [2]:
help(ChatOpenAI)

Help on class ChatOpenAI in module langchain.chat_models.openai:

class ChatOpenAI(langchain.chat_models.base.BaseChatModel)
 |  ChatOpenAI(*, cache: Optional[bool] = None, verbose: bool = None, callbacks: Union[List[langchain.callbacks.base.BaseCallbackHandler], langchain.callbacks.base.BaseCallbackManager, NoneType] = None, callback_manager: Optional[langchain.callbacks.base.BaseCallbackManager] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, client: Any = None, model: str = 'gpt-3.5-turbo', temperature: float = 0.7, model_kwargs: Dict[str, Any] = None, openai_api_key: Optional[str] = None, openai_api_base: Optional[str] = None, openai_organization: Optional[str] = None, openai_proxy: Optional[str] = None, request_timeout: Union[float, Tuple[float, float], NoneType] = None, max_retries: int = 6, streaming: bool = False, n: int = 1, max_tokens: Optional[int] = None, tiktoken_model_name: Optional[str] = None) -> None
 |  
 |  Wrapper around OpenAI C

In [3]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

+ memory is the parameter that we need to change
+ verbose is a parameter that gives us more information about the process.

In [4]:
conversation.predict(input="Hi, my name is Christian")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Christian
AI:

> Finished chain.


"Hello Christian! It's nice to meet you. How can I assist you today?"

In [5]:
conversation.predict(input="Where is Colombia?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Christian
AI: Hello Christian! It's nice to meet you. How can I assist you today?
Human: Where is Colombia?
AI:

> Finished chain.


'Colombia is a country located in the northwestern part of South America. It is bordered by Panama to the northwest, Venezuela and Brazil to the east, and Ecuador and Peru to the south. Colombia also has coastlines along the Caribbean Sea to the north and the Pacific Ocean to the west. The capital city of Colombia is Bogotá.'

Note that how the conversation has been saved.

In [6]:
conversation.predict(input="What is my name?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Christian
AI: Hello Christian! It's nice to meet you. How can I assist you today?
Human: Where is Colombia?
AI: Colombia is a country located in the northwestern part of South America. It is bordered by Panama to the northwest, Venezuela and Brazil to the east, and Ecuador and Peru to the south. Colombia also has coastlines along the Caribbean Sea to the north and the Pacific Ocean to the west. The capital city of Colombia is Bogotá.
Human: What is my name?
AI:

> Finished chain.


'Your name is Christian.'

Now we have a persistence of the name during the conversation

In [7]:
print(memory.buffer)

Human: Hi, my name is Christian
AI: Hello Christian! It's nice to meet you. How can I assist you today?
Human: Where is Colombia?
AI: Colombia is a country located in the northwestern part of South America. It is bordered by Panama to the northwest, Venezuela and Brazil to the east, and Ecuador and Peru to the south. Colombia also has coastlines along the Caribbean Sea to the north and the Pacific Ocean to the west. The capital city of Colombia is Bogotá.
Human: What is my name?
AI: Your name is Christian.


In [8]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Christian\nAI: Hello Christian! It's nice to meet you. How can I assist you today?\nHuman: Where is Colombia?\nAI: Colombia is a country located in the northwestern part of South America. It is bordered by Panama to the northwest, Venezuela and Brazil to the east, and Ecuador and Peru to the south. Colombia also has coastlines along the Caribbean Sea to the north and the Pacific Ocean to the west. The capital city of Colombia is Bogotá.\nHuman: What is my name?\nAI: Your name is Christian."}

lets make another memory instance and how we can add specific things. 

In [9]:
memory_two = ConversationBufferMemory()


In [10]:
memory_two.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [11]:
print(memory_two.buffer)

Human: Hi
AI: What's up


In [12]:
memory_two.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

And we can add new things to the last instance. 

In [13]:
memory_two.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [14]:
memory_two.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

In [15]:
type(memory_two.load_memory_variables({}))

dict

In [17]:
memory_two.load_memory_variables({})['history']

"Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"

Note that the memory is a str in a dict that is persistenced by the key 'history'

## ConversationBufferWindowMemory

This class provides a window of memory, where we can define the parameter k that allows us to save the number of conversational exchages (User - AI). One of the benefits from this class is to save the most recent conversation. 

In [19]:
from langchain.memory import ConversationBufferWindowMemory

In [20]:
memory = ConversationBufferWindowMemory(k=1)               

In [21]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [22]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

Note that only the last part was saved in memory

In [27]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [28]:
conversation.predict(input="Hi, my name is Christian")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Christian
AI:

> Finished chain.


"Hello Christian! It's nice to meet you. How can I assist you today?"

In [29]:
conversation.predict(input="Where is Colombia?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Christian
AI: Hello Christian! It's nice to meet you. How can I assist you today?
Human: Where is Colombia?
AI:

> Finished chain.


'Colombia is a country located in the northwestern part of South America. It is bordered by Panama to the northwest, Venezuela and Brazil to the east, and Ecuador and Peru to the south. Colombia also has coastlines along the Caribbean Sea to the north and the Pacific Ocean to the west. The capital city of Colombia is Bogotá.'

In [30]:
conversation.predict(input="What is my name?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Where is Colombia?
AI: Colombia is a country located in the northwestern part of South America. It is bordered by Panama to the northwest, Venezuela and Brazil to the east, and Ecuador and Peru to the south. Colombia also has coastlines along the Caribbean Sea to the north and the Pacific Ocean to the west. The capital city of Colombia is Bogotá.
Human: What is my name?
AI:

> Finished chain.


"I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation."

Using verbose we can see that the name was not saved in memory because that was used during the first step.

## ConversationTokenBufferMemory

This class allows us to control the number of tokens that we want to save in memory.

In [32]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)

In [33]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [34]:
memory.load_memory_variables({})

{'history': 'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory

This type of memory offers us the opportunity to save a summary of the conversation. 

In [41]:
help(ConversationSummaryBufferMemory)

Help on class ConversationSummaryBufferMemory in module langchain.memory.summary_buffer:

class ConversationSummaryBufferMemory(langchain.memory.chat_memory.BaseChatMemory, langchain.memory.summary.SummarizerMixin)
 |  ConversationSummaryBufferMemory(*, human_prefix: str = 'Human', ai_prefix: str = 'AI', llm: langchain.base_language.BaseLanguageModel, prompt: langchain.schema.prompt_template.BasePromptTemplate = PromptTemplate(input_variables=['summary', 'new_lines'], output_parser=None, partial_variables={}, template='Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.\n\nEXAMPLE\nCurrent summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.\n\nNew lines of conversation:\nHuman: Why do you think artificial intelligence is a force for good?\nAI: Because artificial intelligence will help humans reach their full potential.\n\nNew summary:\nThe hum

In [35]:
from langchain.memory import ConversationSummaryBufferMemory

In [36]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [37]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting.'}

In [38]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [39]:
conversation.predict(input="What would be a good demo to show?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting.
Human: What would be a good demo to show?
AI:

> Finished chain.


'A good demo to show during the lunch meeting with the customer interested in AI would be the latest LLM (Language Model) demo. The LLM is a cutting-edge AI model that can generate human-like text based on a given prompt. It has been trained on a vast amount of data and can generate coherent and contextually relevant responses. By showcasing the LLM demo, you can demonstrate the capabilities of our AI technology and how it can be applied to various industries and use cases.'

In [40]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings and discuss the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting. The human asks what would be a good demo to show, and the AI suggests showcasing the latest LLM (Language Model) demo. The LLM is a cutting-edge AI model that can generate human-like text based on a given prompt. It has been trained on a vast amount of data and can generate coherent and contextually relevant responses. By showcasing the LLM demo, the AI believes they can demonstrate the capabilities of their AI technology and how it can be applied to various industries and use cases.'}